In [19]:
import pandas as pd
import re
import requests
import numpy as np
from bs4 import BeautifulSoup
from glob import iglob

state_df = pd.read_excel(open(next(iglob('Community_Profile_Report*.xlsx')),'rb'), sheet_name='States')

# make first row as header
state_df.columns = state_df.iloc[0]
state_df = state_df[1:]

state_df = state_df.rename(columns={"People with at least 1 dose as % of total population": "People initiating vaccination as % of total population", 
                   "People who are fully vaccinated as % of total population": "People with full course administered as % of total population"})

state_df = state_df[['State', 'State Abbreviation', 'Doses distributed', 'Doses distributed per 100k population',
       'Doses administered', 'Doses administered per 100k population',
       'People initiating vaccination as % of total population',
       'People with full course administered as % of total population',
         'Cumulative cases', 'Cumulative deaths',
         'Viral (RT-PCR) lab test positivity rate - last 7 days (may be an underestimate due to delayed reporting)']]

state_df.to_csv("cdcCovidDataByState.csv", index=False)

# ///county data

df = pd.read_excel(open(next(iglob('Community_Profile_Report*.xlsx')),'rb'), sheet_name='Counties')

# make first row as header
df.columns = df.iloc[0]
df = df[1:]

# drop unallocated columns
df=df.drop(df[df.County.str.contains("Unallocated")].index)

# add zero to 4-digit fips and convert to string for GeoJson join
df["FIPS code"] = df["FIPS code"].astype(str).str.zfill(5)

df=df.rename(columns={"FIPS code" : "county_fips"})

df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('SustainedHotspot', 'Sustained Hotspot')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('Hotspot', 'Hotspot')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('HighBurdenResolving', 'High Burden Resolving')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('ModerateBurdenResolving', 'Moderate Burden Resolving')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('EmergingHotspot', 'Emerging Hotspot')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('ModerateBurden', 'Moderate Burden')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('LowBurden', 'Low Burden')

# df.head()

df= df[['County', 'county_fips',"State Abbreviation",'Cases per 100k - last 7 days', 'Cases - last 7 days', 'Deaths - last 7 days',
 'Deaths per 100k - last 7 days', 'Cases - % change', 'Deaths - % change', 'Cumulative cases','Cumulative deaths',
 'Number of days of downward case trajectory', 'Area of Concern Category', 'Rapid Riser Category', 
'Viral (RT-PCR) lab test positivity rate - last 7 days (may be an underestimate due to delayed reporting)',
 'Confirmed COVID-19 admissions - last 7 days', 'RT-PCR tests per 100k - last 7 days (may be an underestimate due to delayed reporting)',
 'Area of Concern Category', '% inpatient beds occupied', '% staffed adult ICU beds occupied', 
 '% inpatient beds occupied by COVID-19 patient', '% staffed adult ICU beds occupied by COVID-19 patient']]
#save the file to be uploaded to github
df.to_csv("latest-cdc-covid-data-by-county.csv", index=False)